In [ ]:
''' Example usage of the alignment methods in this directory:

  # get your two data sets somehow
  X,Y = gen_data(1000)

  # make correspondences and/or neighbor graphs
  corr = Correspondence(matrix=numpy.eye(1000))
  Wx = neighbor_graph(X,k=5)
  Wy = neighbor_graph(Y,k=5)

  # a linear projector:
  proj = Procrustes(X,Y,corr,2)  # for 2-d output
  Xnew, Ynew = proj.project(X,Y) # or any other data you want to project

  # a non-linear projector:
  Xnew,Ynew = manifold_nonlinear(X,Y,corr,2,Wx,Wy)

  # show the alignment:
  show_alignment(Xnew,Ynew,'nonlinear manifold aln')()
  # or just use normal pyplot functions
'''
import numpy as np
from matplotlib import pyplot

from alignment import (
    TrivialAlignment, Affine, Procrustes, CCA, CCAv2,
    ManifoldLinear, manifold_nonlinear)
from correspondence import Correspondence
from distance import SquaredL2
from neighborhood import neighbor_graph
from synthetic_data import swiss_roll, add_noise, spiral
from util import pairwise_error, Timer
from viz import show_alignment
from warping import (
    ctw, dtw, manifold_warping_linear, manifold_warping_nonlinear)


def gen_data(n, three_d=False):
  t = np.linspace(0,5,n)
  if three_d:
    X = swiss_roll(t,lambda A: np.sin(A)**2)
    Y = np.vstack((np.sin(t)**2,t,np.zeros(n))).T
  else:
    X = spiral(t)
    Y = X[:,(1,0)]  # swap x and y axes
  return add_noise(X,0.05), add_noise(Y,0.05)


if __name__ == '__main__':
  n = 500
  d = 3
  X, Y = gen_data(n, d == 3)
  corr = Correspondence(matrix=np.eye(n))
  Wx = neighbor_graph(X,k=5)
  Wy = neighbor_graph(Y,k=5)

  lin_aligners = (
    ('no alignment',     lambda: TrivialAlignment(X,Y)),
    ('affine',           lambda: Affine(X,Y,corr,d)),
    ('procrustes',       lambda: Procrustes(X,Y,corr,d)),
    ('cca',              lambda: CCA(X,Y,corr,d)),
    ('cca_v2',           lambda: CCAv2(X,Y,d)),
    ('linear manifold',  lambda: ManifoldLinear(X,Y,corr,d,Wx,Wy)),
    ('ctw',              lambda: ctw(X,Y,d)[1]),
    ('manifold warping', lambda: manifold_warping_linear(X,Y,d,Wx,Wy)[1]),
  )

  other_aligners = (
    ('dtw', lambda: (X, dtw(X,Y).warp(X))),
    ('nonlinear manifold aln',
     lambda: manifold_nonlinear(X,Y,corr,d,Wx,Wy)),
    ('nonlinear manifold warp',
     lambda: manifold_warping_nonlinear(X,Y,d,Wx,Wy)[1:]),
  )

  for name, aln in lin_aligners:
    pyplot.figure()
    with Timer(name):
      Xnew,Ynew = aln().project(X, Y)
    print(' sum sq. error =', pairwise_error(Xnew, Ynew, metric=SquaredL2))
    show_alignment(Xnew,Ynew,name)

  for name, aln in other_aligners:
    pyplot.figure()
    with Timer(name):
      Xnew,Ynew = aln()
    print(' sum sq. error =', pairwise_error(Xnew, Ynew, metric=SquaredL2))
    show_alignment(Xnew, Ynew, name)

  pyplot.show()


In [ ]:
from mvlearn.datasets import load_UCImultifeature

In [ ]:
# Load dataset along with labels for digits 0 through 4
n_class = 2
data, labels = load_UCImultifeature(select_labeled = list(range(n_class)))

# Just get the first two views of data
m_data = data[:2]

In [ ]:
m_data

In [ ]:
labels

In [ ]:
# if __name__ == '__main__':
#   n = 400
#   d = 3
#   X, Y = m_data[0], m_data[1][:,:76]
#   corr = Correspondence(matrix=np.eye(n))
#   Wx = neighbor_graph(X,k=5)
#   Wy = neighbor_graph(Y,k=5)

#   lin_aligners = (
#     ('no alignment',     lambda: TrivialAlignment(X,Y)),
# #     ('affine',           lambda: Affine(X,Y,corr,d)),
# #     ('procrustes',       lambda: Procrustes(X,Y,corr,d)),
# #     ('cca',              lambda: CCA(X,Y,corr,d)),
# #     ('cca_v2',           lambda: CCAv2(X,Y,d)),
#     ('linear manifold',  lambda: ManifoldLinear(X,Y,corr,d,Wx,Wy)),
# #     ('ctw',              lambda: ctw(X,Y,d)[1]),
# #     ('manifold warping', lambda: manifold_warping_linear(X,Y,d,Wx,Wy)[1]),
#   )

#   other_aligners = (
# #     ('dtw', lambda: (X, dtw(X,Y).warp(X))),
#     ('nonlinear manifold aln',
#      lambda: manifold_nonlinear(X,Y,corr,d,Wx,Wy)),
# #     ('nonlinear manifold warp',
# #      lambda: manifold_warping_nonlinear(X,Y,d,Wx,Wy)[1:]),
#   )

#   for name, aln in lin_aligners:
#     pyplot.figure()
#     with Timer(name):
#       Xnew,Ynew = aln().project(X, Y)
#     print(' sum sq. error =', pairwise_error(Xnew, Ynew, metric=SquaredL2))
# #     show_alignment(Xnew,Ynew,name)

#   for name, aln in other_aligners:
#     pyplot.figure()
#     with Timer(name):
#       Xnew,Ynew = aln()
#     print(' sum sq. error =', pairwise_error(Xnew, Ynew, metric=SquaredL2))
# #     show_alignment(Xnew, Ynew, name)

#   pyplot.show()

In [ ]:
data_new = np.concatenate((Xnew, Ynew), axis=0)

In [ ]:
# import scipy as sp
# distance_matrix = sp.spatial.distance_matrix(data_new, data_new)

In [ ]:
# distance_matrix

In [ ]:
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.neural_network import MLPClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC
# from sklearn.gaussian_process import GaussianProcessClassifier
# from sklearn.gaussian_process.kernels import RBF
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
# from sklearn.naive_bayes import GaussianNB
# from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

# names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
#          "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
#          "Naive Bayes", "QDA"]

# classifiers = [
#     KNeighborsClassifier(3),
#     SVC(kernel="linear", C=0.025),
#     SVC(gamma=2, C=1),
#     GaussianProcessClassifier(1.0 * RBF(1.0)),
#     DecisionTreeClassifier(max_depth=5),
#     RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
#     MLPClassifier(alpha=1, max_iter=1000),
#     AdaBoostClassifier(),
#     GaussianNB(),
#     QuadraticDiscriminantAnalysis()]

# X, y = m_data[1], labels
# linearly_separable = (X, y)

# datasets = [
# #             make_moons(noise=0.3, random_state=0),
# #             make_circles(noise=0.2, factor=0.5, random_state=1),
#             linearly_separable
#             ]

# # figure = plt.figure(figsize=(27, 9))
# i = 1
# score = []
# # iterate over datasets
# for ds_cnt, ds in enumerate(datasets):
#     # preprocess dataset, split into training and test part
#     X, y = ds
# #     X = StandardScaler().fit_transform(X)
#     X_train, X_test, y_train, y_test = \
#         train_test_split(X, y, test_size=.4, random_state=42)

# #     x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
# #     y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
# #     xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
# #                          np.arange(y_min, y_max, h))

# #     # just plot the dataset first
# #     cm = plt.cm.RdBu
# #     cm_bright = ListedColormap(['#FF0000', '#0000FF'])
# #     ax = plt.subplot(len(datasets), len(classifiers) + 1, i)
# #     if ds_cnt == 0:
# #         ax.set_title("Input data")
# #     # Plot the training points
# #     ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright,
# #                edgecolors='k')
# #     # Plot the testing points
# #     ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright, alpha=0.6,
# #                edgecolors='k')
# #     ax.set_xlim(xx.min(), xx.max())
# #     ax.set_ylim(yy.min(), yy.max())
# #     ax.set_xticks(())
# #     ax.set_yticks(())
#     i += 1

#     # iterate over classifiers
#     for name, clf in zip(names, classifiers):
# #         ax = plt.subplot(len(datasets), len(classifiers) + 1, i)
#         clf.fit(X_train, y_train)
#         score.append(clf.score(X_test, y_test))
# #         score = clf.score(X_test, y_test)

# #         # Plot the decision boundary. For that, we will assign a color to each
# #         # point in the mesh [x_min, x_max]x[y_min, y_max].
# #         if hasattr(clf, "decision_function"):
# #             Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
# #         else:
# #             Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]

# #         # Put the result into a color plot
# #         Z = Z.reshape(xx.shape)
# #         ax.contourf(xx, yy, Z, cmap=cm, alpha=.8)

# #         # Plot the training points
# #         ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright,
# #                    edgecolors='k')
# #         # Plot the testing points
# #         ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright,
# #                    edgecolors='k', alpha=0.6)

# #         ax.set_xlim(xx.min(), xx.max())
# #         ax.set_ylim(yy.min(), yy.max())
# #         ax.set_xticks(())
# #         ax.set_yticks(())
# #         if ds_cnt == 0:
# #             ax.set_title(name)
# #         ax.text(xx.max() - .3, yy.min() + .3, ('%.2f' % score).lstrip('0'),
# #                 size=15, horizontalalignment='right')
#         i += 1

# # plt.tight_layout()
# # plt.show()

In [ ]:
# score

In [ ]:
# score

In [ ]:
# score

In [ ]:
if __name__ == '__main__':
  n = 76
  d = 10
  X, Y = m_data[0].T, m_data[1][:,:76].T
  corr = Correspondence(matrix=np.eye(n))
  Wx = neighbor_graph(X,k=5)
  Wy = neighbor_graph(Y,k=5)

  lin_aligners = (
    ('no alignment',     lambda: TrivialAlignment(X,Y)),
    ('affine',           lambda: Affine(X,Y,corr,d)),
#     ('procrustes',       lambda: Procrustes(X,Y,corr,d)),
#     ('cca',              lambda: CCA(X,Y,corr,d)),
#     ('cca_v2',           lambda: CCAv2(X,Y,d)),
#     ('linear manifold',  lambda: ManifoldLinear(X,Y,corr,d,Wx,Wy)),
#     ('ctw',              lambda: ctw(X,Y,d)[1]),
#     ('manifold warping', lambda: manifold_warping_linear(X,Y,d,Wx,Wy)[1]),
  )

  other_aligners = (
#     ('dtw', lambda: (X, dtw(X,Y).warp(X))),
#     ('nonlinear manifold aln',
#      lambda: manifold_nonlinear(X,Y,corr,d,Wx,Wy)),
#     ('nonlinear manifold warp',
#      lambda: manifold_warping_nonlinear(X,Y,d,Wx,Wy)[1:]),
  )

  for name, aln in lin_aligners:
    pyplot.figure()
    with Timer(name):
      Xnew,Ynew = aln().project(X, Y)
    print(' sum sq. error =', pairwise_error(Xnew, Ynew, metric=SquaredL2))
#     show_alignment(Xnew,Ynew,name)

  for name, aln in other_aligners:
    pyplot.figure()
    with Timer(name):
      Xnew,Ynew = aln()
    print(' sum sq. error =', pairwise_error(Xnew, Ynew, metric=SquaredL2))
#     show_alignment(Xnew, Ynew, name)

  pyplot.show()

In [ ]:
feature_new = np.concatenate((Xnew, Ynew), axis=0)

In [ ]:
feature_new.shape

In [ ]:
import scipy as sp

distance_matrix = sp.spatial.distance_matrix(feature_new, data_new)

In [ ]:
distance_matrix

In [ ]:
sim_mat = 1/(1+distance_matrix)

In [ ]:
sim_mat

In [ ]:
# names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
#          "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
#          "Naive Bayes", "QDA"]

# classifiers = [
#     KNeighborsClassifier(3),
#     SVC(kernel="linear", C=0.025),
#     SVC(gamma=2, C=1),
#     GaussianProcessClassifier(1.0 * RBF(1.0)),
#     DecisionTreeClassifier(max_depth=5),
#     RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
#     MLPClassifier(alpha=1, max_iter=1000),
#     AdaBoostClassifier(),
#     GaussianNB(),
#     QuadraticDiscriminantAnalysis()]

# X, y = m_data[1], labels
# linearly_separable = (X, y)

# datasets = [

#             linearly_separable
#             ]

# i = 1
# score = []
# # iterate over datasets
# for ds_cnt, ds in enumerate(datasets):
#     # preprocess dataset, split into training and test part
#     X, y = ds
# #     X = StandardScaler().fit_transform(X)
#     X_train, X_test, y_train, y_test = \
#         train_test_split(X, y, test_size=.4, random_state=42)

#     i += 1

#     # iterate over classifiers
#     for name, clf in zip(names, classifiers):
# #         ax = plt.subplot(len(datasets), len(classifiers) + 1, i)
#         clf.fit(X_train, y_train)
#         score.append(clf.score(X_test, y_test))

#         i += 1

In [ ]:
# score

In [ ]:
# score

In [ ]:
# score

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim

In [ ]:
class Net(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(152, 50)
        self.relu1 = nn.ReLU()
        self.dout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(50, 100)
        self.prelu = nn.PReLU(1)
        self.out = nn.Linear(100, 1)
        self.out_act = nn.Sigmoid()
        
    def forward(self, input_):
        a1 = self.fc1(input_)
        h1 = self.relu1(a1)
        dout = self.dout(h1)
        a2 = self.fc2(dout)
        h2 = self.prelu(a2)
        a3 = self.out(h2)
        y = self.out_act(a3)
        return y
    
net = Net()
opt = optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999))
criterion = nn.BCELoss()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(np.concatenate((m_data[0], m_data[1][:,:76]), axis=1),
                                                    np.reshape(labels, (-1, 1)),
      test_size=0.05, random_state=73)

# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
#       test_size=0.20, random_state=73)

X_train, y_train = map(
      torch.tensor, (X_train, y_train)
)

# X = torch.from_numpy(
#     np.concatenate((m_data[0], m_data[1][:,:76]), axis=1)
# )
# # Y = torch.from_numpy(
# #     np.concatenate((labels, labels), axis=0)
# # ).view(800, 1)
# Y = torch.from_numpy(
#     labels
# ).view(400, 1)

In [ ]:
X_train.shape

In [ ]:
sim = torch.from_numpy(sim_mat)

In [ ]:
sim = sim.fill_diagonal_(0).float()

In [ ]:
sim.shape

In [ ]:
y_train.shape

In [ ]:
X.size(0)

In [ ]:
def train_epoch(model, opt, criterion, sim, batch_size=50):
    model.train()
    sim = sim
    losses = []
    for beg_i in range(0, X_train.size(0), batch_size):
        x_batch = X_train[beg_i:beg_i + batch_size, :]
        y_batch = y_train[beg_i:beg_i + batch_size, :]
        x_batch = Variable(x_batch)
        y_batch = Variable(y_batch)

        opt.zero_grad()
        # (1) Forward
        y_hat = net(x_batch.float())
        # (2) Compute diff
        loss = criterion(y_hat, y_batch.float())
        reg = torch.tensor(0., requires_grad=True)
        for name, param in net.fc1.named_parameters():
            if 'weight' in name:
#                 reg = torch.norm(reg + param @ sim.float() @ param.T, 2)
                M = .5 * ((torch.eye(152) - sim).T @ (torch.eye(152) - sim)) + .5 * torch.eye(152)
                reg = torch.norm(reg + param @ M.float() @ param.T, 2)
                loss += reg
#         for param in model.parameters():
#             loss += .1 * torch.sum(torch.abs(param))
        # (3) Compute gradients
        loss.backward()
        # (4) update weights
        opt.step()        
        losses.append(loss.data.numpy())
    return losses

In [ ]:
reg = torch.tensor(0., requires_grad=True)
for name, param in net.fc1.named_parameters():
    if 'weight' in name:
        reg = reg + param @ distance_matrix @ param.T

In [ ]:
e_losses = []
num_epochs = 20
for e in range(num_epochs):
    e_losses += train_epoch(net, opt, criterion, sim)

In [ ]:
pyplot.plot(e_losses)

In [ ]:
pyplot.plot(e_losses)

In [ ]:
1+1

In [ ]:
from sklearn.metrics import balanced_accuracy_score

with torch.no_grad():
    x_tensor_test = torch.from_numpy(X_test).float()#.to(device)
    net.eval()
    yhat = net(x_tensor_test)
    y_hat_class = np.where(yhat.cpu().numpy()<0.5, 0, 1)
    test_accuracy = balanced_accuracy_score(y_test.reshape(-1,1), y_hat_class)
print("Test Accuracy {:.2f}".format(test_accuracy))

In [ ]:
from sklearn.metrics import balanced_accuracy_score

with torch.no_grad():
    x_tensor_test = torch.from_numpy(X_test).float()#.to(device)
    net.eval()
    yhat = net(x_tensor_test)
    y_hat_class = np.where(yhat.cpu().numpy()<0.5, 0, 1)
    test_accuracy = balanced_accuracy_score(y_test.reshape(-1,1), y_hat_class)
print("Test Accuracy {:.2f}".format(test_accuracy))